In [2]:
# ===== Reduced Order Modeling with NGSolve + pyMOR (POD-Galerkin) =====
from netgen.occ import *

import numpy as np
from ngsolve import *
from ngsolve import Mesh
from netgen.geom2d import unit_square

from pymor.algorithms.pod import pod
from pymor.vectorarrays.numpy import NumpyVectorSpace
import matplotlib.pyplot as plt
from collections import Counter


In [7]:
# ---------- Parameters ----------
L, W, H      = 3.0, 1.0, 1.0
baffle_thk   = 0.05
baffle_len   = 0.70*W
baffle_x     = [0.7, 1.5, 2.3]   # positions of the three baffles
half_model   = False
maxh         = 0.1

# stub parameters
Lin_ext  = 0.4   # inlet stub length
Lout_ext = 0.4   # outlet stub length
stub_frac = 0.5  # fraction of duct cross-section used for stub

# ---------- Main duct ----------
duct = Box(Pnt(0,0,0), Pnt(L, W, H))
duct.faces.name = "walls"

# ---------- Baffles ----------
fluid = duct
for i, x0 in enumerate(baffle_x, 1):
    x0min = x0 - 0.5*baffle_thk
    x0max = x0 + 0.5*baffle_thk
    zmin, zmax = 0, H

    if i == 2:  # middle baffle from TOP wall
        ymin, ymax = W - baffle_len, W
    else:       # side baffles from BOTTOM wall
        ymin, ymax = 0, baffle_len

    plate = Box(Pnt(x0min, ymin, zmin), Pnt(x0max, ymax, zmax))
    plate.faces.name = f"baffle{i}"
    fluid = fluid - plate

# ---------- Inlet stub ----------
stub_ymin = (1-stub_frac)/2 * W
stub_ymax = (1+stub_frac)/2 * W
stub_zmin = (1-stub_frac)/2 * H
stub_zmax = (1+stub_frac)/2 * H

stub_in = Box(Pnt(-Lin_ext, stub_ymin, stub_zmin),
              Pnt(0,        stub_ymax, stub_zmax))
stub_in.faces.name = "walls"           # stub sides are walls
stub_in.faces.Min(X).name = "inlet"    # only far end is inlet

# ---------- Outlet stub ----------
stub_out = Box(Pnt(L, stub_ymin, stub_zmin),
               Pnt(L+Lout_ext, stub_ymax, stub_zmax))
stub_out.faces.name = "walls"          # stub sides are walls
stub_out.faces.Max(X).name = "outlet"  # only far end is outlet

# ---------- Combine duct + stubs ----------
fluid = fluid + stub_in + stub_out

# ---------- Apply half-model cut if desired ----------
if half_model:
    sym_half = HalfSpace(Pnt(0, W/2, 0), Vec(0,1,0))
    fluid = fluid * sym_half

# ---------- Build OCC geometry and mesh ----------
geo = OCCGeometry(fluid)
m = geo.GenerateMesh(maxh=maxh)
mesh = Mesh(m)

print("Mesh elements:", mesh.ne)
print("Boundary names:", mesh.GetBoundaries())

# ---------- Count boundary elements ----------
counter = Counter()
for el in mesh.Elements(BND):
    counter[el.mat] += 1
print("Boundary facet counts:", counter)

# ---------- Visualize boundaries ----------
cf = mesh.BoundaryCF({
    "inlet":   10,
    "outlet":  20,
    "walls":   30,
    "baffle1": 40,
    "baffle2": 50,
    "baffle3": 60
}, default=0)

Draw(mesh)

# Draw(cf, mesh, draw_vol=False, draw_surf=True, order=3,
#      clipping=False, name="Boundary map")


Mesh elements: 16210
Boundary names: ('walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'walls', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1', 'inlet', 'walls', 'walls', 'walls', 'walls', 'outlet')
Boundary facet counts: Counter({'walls': 3324, 'baffle2': 374, 'baffle1': 336, 'baffle3': 318, 'inlet': 56, 'outlet': 50})


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [13]:
from ngsolve import *
from ngsolve import x,y,z
from ngsolve.webgui import Draw

SetNumThreads(1)   # helps avoid random Windows kernel crashes

# ----------------- Navier–Stokes (Stokes warm start + Oseen) -----------------
rho  = 1.0
nu_f = 1e-2      # fluid viscosity; increase if Oseen struggles
U0   = 1.0       # inlet speed scale

# Taylor–Hood mixed space
V = VectorH1(mesh, order=2, dirichlet="walls|baffle1|baffle2|baffle3|inlet")
Q = H1(mesh, order=1)
fes = V*Q
(u,p), (v,q) = fes.TnT()

# inlet profile (parabolic in y,z)
uin = U0*(1 - (2*y/W-1)**2)*(1 - (2*z/H-1)**2)
u_inlet = CoefficientFunction((uin,0,0))

# lifting in mixed space: velocity BC only
g = GridFunction(fes)
g.components[0].Set(u_inlet, BND, definedon=mesh.Boundaries("inlet"))
g.components[0].Set((0,0,0),  BND, definedon=mesh.Boundaries("walls|baffle1|baffle2|baffle3"))

# ----- 3.1 Stokes solve (no convection) -----
aS = BilinearForm(fes, symmetric=True)
aS += rho*nu_f*InnerProduct(Grad(u),Grad(v))*dx
aS += (-div(v)*p - q*div(u))*dx
aS += 1e-12*p*q*dx   # pressure pin

fS = LinearForm(fes) # no body force

with TaskManager():
    aS.Assemble(); fS.Assemble()
    rhs = fS.vec.CreateVector()
    rhs.data = fS.vec - aS.mat*g.vec
    u_p = GridFunction(fes)
    invS = aS.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
    u_p.vec.data = g.vec + invS*rhs

# extract velocity, pressure
u_vel = u_p.components[0]
p_prs = u_p.components[1]

# ----- 3.2 Oseen (Picard) steps -----
uk = GridFunction(V); uk.vec.data = u_vel.vec  # convecting velocity
n_oseen = 3                                    # a few iterations are enough

for it in range(n_oseen):
    aO = BilinearForm(fes, symmetric=False)
    aO += rho*nu_f*InnerProduct(Grad(u),Grad(v))*dx
    aO += rho*InnerProduct(Grad(u)*uk, v)*dx
    aO += (-div(v)*p - q*div(u))*dx
    aO += 1e-12*p*q*dx

    fO = LinearForm(fes)

    with TaskManager():
        aO.Assemble(); fO.Assemble()
        rhs = fO.vec.CreateVector()
        rhs.data = fO.vec - aO.mat*g.vec
        sol = GridFunction(fes)
        invO = aO.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        sol.vec.data = g.vec + invO*rhs

    u_vel.vec.data = sol.components[0].vec
    p_prs.vec.data = sol.components[1].vec
    uk.vec.data    = u_vel.vec
    print(f"Oseen iter {it+1}: ||u|| = {Norm(u_vel.vec):.3e}")

# convection field for scalar problem:
b = CoefficientFunction((u_vel[0], u_vel[1], u_vel[2]))

# ----------------- Scalar advection–diffusion (Eq. 3.88) -----------------
# parameters
g1, g2, g3 = 0.0, 20.0, 50.0
Pe         = 100.0
nu_s       = 1.0/Pe   # scalar diffusivity

Vs = H1(mesh, order=1, dirichlet="walls|inlet|baffle1|baffle2|baffle3")
us, vs = Vs.TnT()

a = BilinearForm(Vs, symmetric=False)
a += nu_s*InnerProduct(grad(us), grad(vs))*dx
a += (b*grad(us))*vs*dx

# SUPG (optional; start with it OFF; enable if you see oscillations)
# from ngsolve import specialcf, Norm, IfPos, sinh, cosh, Id
# h  = specialcf.mesh_size
# bn = Norm(b)+1e-12
# PeK= bn*h/(2*nu_s)
# coth = lambda x: (cosh(x)+1e-12)/(sinh(x)+1e-12)
# tau  = IfPos(PeK-1e-6, h/(2*bn)*(coth(PeK)-1/PeK), h*h/(12*nu_s))
# a += tau*(b*grad(us))*(b*grad(vs))*dx
# # optional tiny cross-wind diffusion:
# bt = b/bn; Pperp = Id(3) - OuterProduct(bt,bt); eps_cw = 0.03*bn*h
# a += eps_cw*InnerProduct(Pperp*grad(us), Pperp*grad(vs))*dx

f = LinearForm(Vs)
with TaskManager():
    f.Assemble()

# Dirichlet lifting for scalar
gT = GridFunction(Vs)
gT.Set(CoefficientFunction(0.0), BND, definedon=mesh.Boundaries("walls|inlet"))
gT.Set(CoefficientFunction(g1),  BND, definedon=mesh.Boundaries("baffle1"))
gT.Set(CoefficientFunction(g2),  BND, definedon=mesh.Boundaries("baffle2"))
gT.Set(CoefficientFunction(g3),  BND, definedon=mesh.Boundaries("baffle3"))

with TaskManager():
    a.Assemble()
    rhs = f.vec.CreateVector()
    rhs.data = f.vec - a.mat*gT.vec

    uN = GridFunction(Vs)
    uN.vec.data = gT.vec
    inv = a.mat.Inverse(Vs.FreeDofs(), inverse="sparsecholesky")
    uN.vec.data += inv*rhs

print("scalar u range:", float(np.array(uN.vec).min()), "→", float(np.array(uN.vec).max()))

# quick look
Draw(uN, mesh, min=0, max=max(g1,g2,g3), name="u_scalar")
VTKOutput(ma=mesh, coefs=[uN], names=["u"], filename="solution", subdivision=2).Do()


Oseen iter 1: ||u|| = 0.000e+00
Oseen iter 2: ||u|| = 0.000e+00
Oseen iter 3: ||u|| = 0.000e+00
scalar u range: 0.0 → 50.00000000000002


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

'solution'

In [12]:
print(mesh.GetBoundaries())
print(counter)   # your boundary counts


('walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'walls', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1', 'inlet', 'walls', 'walls', 'walls', 'walls', 'outlet')
Counter({'walls': 3324, 'baffle2': 374, 'baffle1': 336, 'baffle3': 318, 'inlet': 56, 'outlet': 50})


In [6]:
# ---- Advection–diffusion PDE from Quarteroni §3.8 (Eq. 3.88) ----
#    -nu Δu + b·∇u = 0
#    u = gp on baffles, u=0 on walls+inlet,
#    Neumann (do-nothing) at outlet

from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve import x, y, z


# ---- parameters ----
g1, g2, g3 = 0.0, 50.0, 0.0    # Dirichlet values on baffle1-3
Pe         = 1.0              # Peclet number (mu4)
nu         = 1.0/Pe            # diffusion coefficient (since V=L=1)

print("approx Pe_K ~", ((1.0*maxh)/(2*nu)))   # if max |b|≈1


# ---- FE space ----
# Dirichlet on walls, inlet, and baffles
V = H1(mesh, order=1,dirichlet="walls|inlet|baffle1|baffle2|baffle3")
u, v = V.TnT()


# ---- convection field b(x) ----
# Poiseuille-like profile in y,z, pointing along x
bx = (1 - (2*y/W-1)**2) * (1 - (2*z/H-1)**2)
b  = CoefficientFunction((bx, 0, 0))

# ---- bilinear and linear forms ----
a = BilinearForm(V, symmetric=False)
a += nu * InnerProduct(grad(u), grad(v)) * dx    # diffusion
a += b * grad(u) * v * dx                        # advection

# optional streamline-diffusion stabilization (SUPG)

hK = specialcf.mesh_size   # local element size coefficient function
tau = 0.2 * hK
a += tau * (b*grad(u)) * (b*grad(v)) * dx

f = LinearForm(V)    # no volumetric source
f.Assemble()

# ---- lifting for Dirichlet BCs: set ALL in one shot (boundary-wise CF) ----
# map boundary-name -> value
bc_vals = {
    "walls":   0.0,    # book: u=0 on walls
    "inlet":   0.0,    # book: u=0 on inlet
    "baffle1": g1,
    "baffle2": g2,
    "baffle3": g3,
    "outlet":  None    # Neumann (natural); no Dirichlet here
}

bnd_names = list(mesh.GetBoundaries())
vals_list = []
for name in bnd_names:
    v = bc_vals.get(name, None)
    vals_list.append(0.0 if v is None else float(v))
cf_bndwise = CoefficientFunction(vals_list)
g = GridFunction(V)
g.Set(cf_bndwise, BND)   # one call, boundary-wise assignment

# sanity check: averages on each baffle (should equal g1,g2,g3)
for name, target in [("baffle1", g1), ("baffle2", g2), ("baffle3", g3)]:
    area = Integrate(1, mesh, BND, definedon=mesh.Boundaries(name))
    avg  = Integrate(g, mesh, BND, definedon=mesh.Boundaries(name)) / area
    print(f"avg g on {name} = {avg} (target {target})")

# ---- assemble and solve ----
a.Assemble()
rhs = f.vec.CreateVector()
rhs.data = f.vec - a.mat * g.vec

uN = GridFunction(V)
uN.vec.data = g.vec    # start with Dirichlet lifting
inv = a.mat.Inverse(V.FreeDofs(), inverse="sparsecholesky")
uN.vec.data += inv * rhs


vals = np.array(uN.vec)
print("min u =", vals.min())
print("max u =", vals.max())

# ---- visualize ----
print(mesh.GetBoundaries())

Draw(uN, mesh, name="u")

from ngsolve import VTKOutput
VTKOutput(ma=mesh, coefs=[uN], names=["u"], filename="solution", subdivision=2).Do()


approx Pe_K ~ 0.05
avg g on baffle1 = 0.0 (target 0.0)
avg g on baffle2 = 46.609818584513235 (target 50.0)
avg g on baffle3 = 0.0 (target 0.0)
min u = 0.0
max u = 50.00000000000001
('walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'walls', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1', 'inlet', 'walls', 'walls', 'walls', 'walls', 'outlet')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

'solution'